In [1]:
from __future__ import absolute_import, division, print_function
# 放在第一句，不然会报错

import logging
from operator import index
import os
import random
from matplotlib.pyplot import title
import numpy as np
import pandas as pd
import csv

import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, Dataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange
from random import randrange, randint, shuffle, choice, sample

from transformers import BertForMaskedLM,BertConfig,BertTokenizer
from transformers import AdamW
from transformers.optimization import (
    get_constant_schedule,
    get_constant_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup,
)

import multi_tasks_pretrain_args as args
import json
from sklearn.model_selection import KFold

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
# Random Mask
def create_masked_lm_predictions(tokens, masked_lm_prob, max_predictions_per_seq, vocab_list):
    """Creates the predictions for the masked LM objective. This is mostly copied from the Google BERT repo, but
    with several refactors to clean it up and remove a lot of unnecessary variables."""
    # 对一个句子进行MLM
    """
    tokens:一句话的tokens
    masked_lm_prob:覆盖的概率
    max_predictions_per_seq:每个seq，最大预测
    vocab_list:词汇表
    return:经过MLM的tokens，mask的index，mask的label
    """
    cand_indices = []
    for (i, token) in enumerate(tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue
        cand_indices.append(i)
    # 获取除了，特殊的CLS和SEP的词语的token下标
    num_to_mask = min(max_predictions_per_seq,
                      max(1, int(round(len(tokens) * masked_lm_prob))))
    # mask数量，极端情况下，至少为1，最多max_predictions_per_seq
    # print(num_to_mask)
    # print("tokens", len(tokens))
    # print("cand", len(cand_indices))
    shuffle(cand_indices)
    # 打乱cand_indices
    mask_indices = sorted(sample(cand_indices, num_to_mask))
    # 从cand_indices随机抽取num_to_mask个元素，并且以list返回，然后进行排序
    masked_token_labels = []
    for index in mask_indices:
        # 80% of the time, replace with [MASK]
        masked_token = "[MASK]"
        masked_token_labels.append(tokens[index])
        # Once we've saved the true label for that token, we can overwrite it with the masked version
        tokens[index] = masked_token

    return tokens, mask_indices, masked_token_labels

In [4]:
def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
    """Truncates a pair of sequences to a maximum sequence length. Lifted from Google's BERT repo."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_num_tokens:
            break

        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()

In [6]:
def Json_File_Reader(data_name,data_paths,dynamic_mask_times=None):
    all_data = []
    for index,data_path in enumerate(data_paths):
            with open(data_path,'r',encoding='utf-8') as f:
                for item in f.readlines():
                    item = json.loads(item)
                    # item['data_type'] = data_name
                    item['is_val'] = index
                    all_data.append(item)
    
    if dynamic_mask_times:
        temp_data = all_data.copy()
        for i in range(dynamic_mask_times-1):
            all_data = all_data + temp_data
    return all_data

In [7]:
class RTE_Processing:
    def __init__(self,data,tokenizer,max_seq_length,hyperparameter_mask,vocab_list):
        self.data = data
        self.data_type = args.superglue['RTE']
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.masked_lm_prob = hyperparameter_mask['masked_lm_prob']
        self.max_predictions_per_seq = hyperparameter_mask['max_predictions_per_seq']
        self.vocab_list = vocab_list
        self.special_token_length = 2
        # [CLS] and [SEP]
        self.hard_prompt_token_length = 8

        self.max_token_length = self.max_seq_length - self.special_token_length-self.hard_prompt_token_length-args.soft_prompt_length

    def Creat_Input_For_PLMs(self):
        
        data_input = []
        for index,item in tqdm(enumerate(self.data),desc="RTE Data Processing"):
            sentence1 = item['premise'].strip('\n')
            sentence2 = item['hypothesis'].strip('\n').strip('.')
            label = item['label']
            label_ids = self.data_type['label_list'].index(label)

            answer = self.data_type['prompt_answer_list'][label_ids]

            tokens_a = self.tokenizer.tokenize(sentence1)
            tokens_b = self.tokenizer.tokenize(sentence2)
            answer_tokens = self.tokenizer.tokenize(answer)

            if item['is_val']:
                answer_tokens = []
                max_token_length = self.max_token_length + 1
            else:
                max_token_length = self.max_token_length 

            truncate_seq_pair(tokens_a, tokens_b, max_token_length)
            tokens = ["[CLS]"] +["[unused{}]".format(i+1) for i in range(args.soft_prompt_length)]+ tokens_a +["Question",":"] + tokens_b +["?","the","Answer",":"]+ answer_tokens + ["."] + ["[SEP]"]

            start = len(tokens_a)+1+args.soft_prompt_length+2+len(tokens_b)+4
            prompt_positions = [i for i in range(start,start+1)]
            
            tokens, masked_lm_positions, masked_lm_labels = create_masked_lm_predictions(tokens, self.masked_lm_prob, self.max_predictions_per_seq, self.vocab_list)

            input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            attention_ids = [1] * len(input_ids)
            segment_ids = [0] * len(input_ids)
            masked_lm_labels = self.tokenizer.convert_tokens_to_ids(masked_lm_labels)
            data_item = {'input_ids':input_ids,
                'segment_ids':segment_ids,
                'attention_ids':attention_ids,
                'masked_lm_positions':masked_lm_positions,
                'masked_lm_labels':masked_lm_labels}
            # no padding
            data_input.append(data_item)

        return data_input

In [8]:
def Divide_Data(data_list,eval_rate):
    """
    data_list: a list contains a dict of item of input for PLMs
    It is not approporite for those task whose contains very few data. In other words, It is unfair.
    """
    kf = KFold(n_splits=int(1/eval_rate),shuffle=True,random_state=args.seed)
    train_data = []
    eval_data = []

    for step,(train_index,eval_index) in enumerate(kf.split(data_list)):
        for i in train_index:
            train_data.append(data_list[i])
        for i in eval_index:
            eval_data.append(data_list[i])
        if step >= 0:
            break
    return train_data,eval_data

In [9]:
class PretrainDataset(Dataset):
    def __init__(self,data,max_seq_length):
        self.max_seq_length = max_seq_length
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        data_item = self.data[index]
        input_ids = data_item['input_ids']
        segment_ids = data_item['segment_ids']
        attention_ids = data_item['attention_ids']
        masked_lm_positions = data_item['masked_lm_positions']
        masked_lm_labels = data_item['masked_lm_labels']

        assert len(input_ids)==len(segment_ids)==len(attention_ids)
        padding_length = self.max_seq_length - len(input_ids)

        input_ids += [0] * padding_length
        attention_ids += [0] * padding_length
        segment_ids += [0] * padding_length
        label_ids = np.full(self.max_seq_length, dtype=np.int, fill_value=-100)
        label_ids[masked_lm_positions] = masked_lm_labels

        input_ids = torch.tensor(input_ids,dtype=torch.long)
        segment_ids = torch.tensor(segment_ids,dtype=torch.long)
        attention_ids = torch.tensor(attention_ids,dtype=torch.long)
        label_ids = torch.tensor(label_ids,dtype=torch.long)
        return input_ids, segment_ids, attention_ids, label_ids

In [31]:
def read_data(data_paths,dynamic_mask_times=None,eval_rate=0.1):
        """
        data_path:文件格式为txt，tsv，csv的文件路径
        第一列或者第一二列为文本，最后一列为标签
        dynamic_mask_times:动态掩词
        eval_rate:验证集比例
        """
        sentence1 = []
        sentence2 = []
        label = []
        ID = []
        # ID is a columns to distinguish train data or val data
        for index,data_path in enumerate(data_paths):
            with open(data_path,'r',encoding='utf-8') as f:
                for item in f.readlines():
                    item = json.loads(item)
                    sentence1.append(item['premise'])
                    sentence2.append(item['hypothesis'])
                    label.append(item['label'])
                    ID.append(index)
            
#         all_data = pd.read_csv(data_paths)
#         all_data = pd.read_csv(data_paths, sep='\t', header=None, quoting=csv.QUOTE_NONE,encoding='utf-8')
        # all_data = pd.DataFrame({'premise':sentence1,'hypothesis':sentence2,'label':label})
        all_data = pd.DataFrame({'premise':sentence1,'hypothesis':sentence2,'label':label,'ID':ID})
        
        if dynamic_mask_times:
            temp_data = all_data.copy()
            for _ in range(dynamic_mask_times-1):
                all_data = pd.concat([all_data,temp_data])

        kf = KFold(n_splits=int(1/eval_rate),shuffle=True,random_state=args.seed)
        for step,(train_index,eval_index) in enumerate(kf.split(all_data)):
            eval_data = all_data.iloc[eval_index]
            train_data = all_data.iloc[train_index]
            if step >= 0:
                break
        return train_data,eval_data

In [32]:
class PretrainDataset(Dataset):
    def __init__(self,DATAFRAME,tokenizer,prompt_pattern_list,label_list,max_seq_length,masked_lm_prob,max_predictions_per_seq,vocab_list):
        self.data = DATAFRAME
        self.columns = list(DATAFRAME.columns)
        """
        DATAFRAME:为pandas的DataFrame，需要第一列文本
        """
        self.tokenizer = tokenizer
        self.prompt_pattern_list = prompt_pattern_list
        self.label_list = label_list
        
        self.max_seq_length = max_seq_length
        self.nums_specical_tokens = 2
#         self.max_tokens_nums = max_seq_length - self.nums_specical_tokens - (2+2)-args.soft_prompt_length
        self.max_tokens_nums = max_seq_length - self.nums_specical_tokens - (8)-args.soft_prompt_length
                                                                             
        # 多了两个逗号
        self.masked_lm_prob = masked_lm_prob
        self.max_predictions_per_seq = max_predictions_per_seq
        self.vocab_list = vocab_list
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self,index):
        text_a = self.data[self.columns[0]].iloc[index].strip('\n')
        text_b = self.data[self.columns[1]].iloc[index].strip('\n')

        label = self.label_list.index(self.data[self.columns[2]].iloc[index])
        ID = self.data[self.columns[3]].iloc[index]
#         label = self.data[self.columns[2]].iloc[index]
        
#         template = choice(self.prompt_pattern_list[label])
        template = self.prompt_pattern_list[label]

#         template = choice(self.prompt_pattern_list[choice([0,1,2])])
        
#         template_label = self.prompt_pattern_list[label]
#         template = self.prompt_pattern[0][0:self.prompt_pattern[1]]+template_label+self.prompt_pattern[0][self.prompt_pattern[1]:]
        
        tokens_template = self.tokenizer.tokenize(template)
        if ID:
            # tokens_template = ["[MASK]"]
            # tokens_template = ["Yes",'or','No']
            tokens_template = []

            # max_tokens_nums = self.max_tokens_nums - 2
            max_tokens_nums = self.max_tokens_nums + 1

        else:
            max_tokens_nums = self.max_tokens_nums
            
        tokens_a = self.tokenizer.tokenize(text_a)
        tokens_b = self.tokenizer.tokenize(text_b)
        truncate_seq_pair(tokens_a, tokens_b, max_tokens_nums)
        # tokens = tokens[:self.max_tokens_nums]

        tokens = ["[CLS]"] +["[unused{}]".format(i+1) for i in range(args.soft_prompt_length)]+ tokens_a +["Question",":"] + tokens_b +["?","The","Answer",":"]+ tokens_template + ["."] + ["[SEP]"]
        # tokens = ["[CLS]"] +["[unused{}]".format(i+1) for i in range(args.soft_prompt_length)]+ tokens_a + tokens_template + [','] + tokens_b+ ["[SEP]"]
        # tokens = ["[CLS]"] +["[unused{}]".format(i+1) for i in range(args.soft_prompt_length)]+ tokens_a + ['，'] + tokens_template + tokens_b+ ["[SEP]"]                                                               
        # masked_labels = tokens.copy()

        start = len(tokens_a)+1+args.soft_prompt_length+2+len(tokens_b)+4
        
        prompt_positions = [i for i in range(start,start+1)]
        
        tokens, masked_lm_positions, masked_lm_labels = create_masked_lm_predictions(tokens, self.masked_lm_prob, self.max_predictions_per_seq, self.vocab_list)

        input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        # label_ids = self.tokenizer.conver_tokens_to_ids(masked_labels)

        attention_ids = [1] * len(input_ids)
        # if ID:
        #     attention_ids[start] = 0
            # we do not use attention on the position of answer for validation data set
        padding_length = self.max_seq_length - len(input_ids)

        input_ids += [0] * padding_length
        # label_ids += [0] * padding_length

        attention_ids += [0] * padding_length

        segment_ids = [0] * len(input_ids)
        label_ids = np.full(self.max_seq_length, dtype=np.int, fill_value=-100)
        label_ids[masked_lm_positions] = self.tokenizer.convert_tokens_to_ids(masked_lm_labels)

        input_ids = torch.tensor(input_ids,dtype=torch.long)
        segment_ids = torch.tensor(segment_ids,dtype=torch.long)
        attention_ids = torch.tensor(attention_ids,dtype=torch.long)
        label_ids = torch.tensor(label_ids,dtype=torch.long)

        return input_ids, segment_ids, attention_ids, label_ids

In [33]:
train_data,eval_data = read_data(args.superglue['RTE']['data_path'],10)

In [34]:
TrainDataset = PretrainDataset(train_data,tokenizer,args.superglue['RTE']['prompt_answer_list'],args.superglue['RTE']['label_list'],128,0.25,20,vocab_list)
EvalDataset = PretrainDataset(eval_data,tokenizer,args.superglue['RTE']['prompt_answer_list'],args.superglue['RTE']['label_list'],128,0.25,20,vocab_list)

In [10]:
if not os.path.exists(args.model_save_path):
    os.makedirs(args.model_save_path)

In [11]:
if args.gradient_accumulation_steps < 1:
    raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
        args.gradient_accumulation_steps))

In [2]:
args.train_batch_size = args.train_batch_size // args.gradient_accumulation_steps
# seed everything,python,numpy and torch

np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

In [51]:
random.seed(42)

In [52]:
random.random()

0.6394267984578837

In [3]:
tokenizer = BertTokenizer.from_pretrained(args.model_name_from_hugging_face)
vocab_list = list(tokenizer.vocab.keys())
config = BertConfig.from_pretrained(args.model_name_from_hugging_face)

In [14]:
RTE_data = Json_File_Reader('RTE',args.superglue['RTE']['data_path'],10)

In [15]:
RTE = RTE_Processing(RTE_data,tokenizer,
                     args.max_seq_length,
                     args.superglue['RTE']['hyperparameter_of_mask'],
                     vocab_list)

In [16]:
RTE_inputs = RTE.Creat_Input_For_PLMs()
# shuffle(RTE_inputs)

RTE Data Processing: 27670it [00:46, 595.65it/s]


In [17]:
train_data,eval_data = Divide_Data(RTE_inputs,0.1)

In [18]:
num_train_optimization_steps = int(len(train_data) / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs

In [19]:
model = BertForMaskedLM.from_pretrained(args.model_name_from_hugging_face)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
device = torch.device("cuda")

In [21]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [22]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
    'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [23]:
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

In [25]:
scheduler = get_linear_schedule_with_warmup(optimizer,num_training_steps=num_train_optimization_steps,num_warmup_steps = num_train_optimization_steps*args.warmup_proportion)

TrainDataset = PretrainDataset(train_data,args.max_seq_length)
EvalDataset = PretrainDataset(eval_data,args.max_seq_length)

TrainDataLoader = DataLoader(TrainDataset,batch_size=args.train_batch_size,shuffle=True)
EvalDataLoader = DataLoader(EvalDataset,batch_size=args.eval_batch_size,shuffle=False)

In [26]:
logger = logging.getLogger(__name__)

In [27]:
global_step = 0
best_loss = 100000
patience = 0

model.train()

for e in trange(int(args.num_train_epochs), desc="Epoch"):
    if patience > args.early_stopping:
        break
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    optimizer.zero_grad()
    for step, batch in enumerate(tqdm(TrainDataLoader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, segment_ids, attention_ids, label_ids = batch
        # masked_lm_loss
        outputs = model(input_ids=input_ids, attention_mask=attention_ids, token_type_ids=segment_ids, labels=label_ids)
        loss = outputs.loss
        if args.gradient_accumulation_steps > 1:
            loss = loss / args.gradient_accumulation_steps

        if args.fp16:
            optimizer.backward(loss)
        else:
            loss.backward()

        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % args.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()

            global_step += 1
        if nb_tr_steps > 0 and nb_tr_steps % 100 == 0:
            logger.info("===================== -epoch %d -train_step %d -train_loss %.4f\n" % (e, nb_tr_steps, tr_loss / nb_tr_steps))
    if nb_tr_steps > 0:
        #################################EVAL#####################################################
        model.eval()
        eval_loss = 0
        nb_eval_steps = 0
        for step, batch in enumerate(tqdm(EvalDataLoader, desc="Evaluating")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, segment_ids, attention_ids, label_ids = batch
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_ids, token_type_ids=segment_ids, labels=label_ids)
                loss = outputs.loss
            eval_loss += loss.item()
            nb_eval_steps += 1
        print(nb_tr_steps)

        print(tr_loss)
        print(nb_eval_steps)
        print(eval_loss)
        eval_loss = eval_loss / nb_eval_steps
        if eval_loss < best_loss:
            patience = 0
            # Save a trained model, configuration and tokenizer
            print("The eval loss is decreasing!,so we save model!")
            model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

            # If we save using the predefined names, we can load using `from_pretrained`
            # output_model_file = os.path.join(model_fold_path, WEIGHTS_NAME)
            output_model_file = os.path.join(args.model_save_path, 'pytorch_model{:.3f}.bin'.format(eval_loss))
            torch.save(model_to_save.state_dict(), output_model_file)
            tokenizer.save_vocabulary(os.path.join(args.model_save_path,'vocab.txt'))
            config.to_json_file(os.path.join(args.model_save_path,'config.json'))
            #torch.save(model.module, output_model_file)

            best_loss = eval_loss

        else:
            patience += 1
        print("============================ -epoch %d -train_loss %.4f -eval_loss %.4f\n"% (e, tr_loss / nb_tr_steps, eval_loss))

    if best_loss<=0.1:
        exit(0)

Iteration:   0%|          | 0/779 [00:00<?, ?it/s]<ipython-input-11-2468ff21c7fc>:83: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  label_ids = np.full(self.max_seq_length, dtype=np.int, fill_value=-100)

Iteration:  32%|███▏      | 246/779 [00:53<01:57,  4.55it/s]


Iteration:  64%|██████▍   | 502/779 [01:48<00:59,  4.65it/s]


Iteration:  97%|█████████▋| 758/779 [02:43<00:04,  4.57it/s]


Iteration: 100%|██████████| 779/779 [02:47<00:00,  4.64it/s]

Evaluating: 100%|██████████| 87/87 [00:09<00:00,  8.94it/s]


779
867.3138144612312
87
158.1931689977646
The eval loss is decreasing!,so we save model!


...

============================ -epoch 13 -train_loss 0.2155 -eval_loss 0.4463




Iteration:  33%|███▎      | 256/779 [00:54<01:54,  4.58it/s]


Iteration:  66%|██████▌   | 512/779 [01:49<00:58,  4.58it/s]


Epoch:  47%|████▋     | 14/30 [43:01<49:10, 184.41s/it]


KeyboardInterrupt: 

In [29]:
model.eval()
eval_loss = 0
nb_eval_steps = 0
for step, batch in enumerate(tqdm(EvalDataLoader, desc="Evaluating")):
    batch = tuple(t.to(device) for t in batch)
    input_ids, segment_ids, attention_ids, label_ids = batch
    if step >=0 :
        break
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_ids, token_type_ids=segment_ids, labels=label_ids)
        loss = outputs.loss
    eval_loss += loss.item()
    nb_eval_steps += 1

Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]<ipython-input-9-7ca3da7e7a4d>:23: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  label_ids = np.full(self.max_seq_length, dtype=np.int, fill_value=-100)
Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]


In [28]:
input_ids[0]

tensor([  101,   103,   125,  1550,  1482,   103,  1138,  1120,  1655,  1141,
         6486,  1150,  2242,  1103,   103,   119,   103,   119, 19241,  1127,
         1255,   103,  1103,   103,  1311,  1105,   103,   158,   119,   156,
          119,  4037,  1112,   170,  1871,   103,   103,  1106,   103,  2025,
         3303,  1118,  1103,   153,  5773,  6098,  1945,   119,  1337,   112,
          188,  1164,   103,  7541,  1104,  1103,  3555,   126,   119,   126,
         1550,  1482,  1104,  5696,   103,  1656,   103,   103,  1311,   117,
         2452,  1106,  1103,  2025,   119,   103,   122,   103,   129,  1550,
         1482,  1104,  5576, 13335, 15447, 16736,  7162,  1686,  1107,   103,
          117,  1103,  2025,  1276,   119, 22171,   131,  2677,  7541,  1104,
          158,   119,   156,   119,  5696,  7162,   103,  1482,   136,   103,
        26018,   103,  1302,   119,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],

In [30]:
input_ids[0]

tensor([  101,   103,   125,  1550,  1482,   103,  1138,  1120,  1655,  1141,
         6486,  1150,  2242,  1103,   103,   119,   103,   119, 19241,  1127,
         1255,   103,  1103,   103,  1311,  1105,   103,   158,   119,   156,
          119,  4037,  1112,   170,  1871,   103,   103,  1106,   103,  2025,
         3303,  1118,  1103,   153,  5773,  6098,  1945,   119,  1337,   112,
          188,  1164,   103,  7541,  1104,  1103,  3555,   126,   119,   126,
         1550,  1482,  1104,  5696,   103,  1656,   103,   103,  1311,   117,
         2452,  1106,  1103,  2025,   119,   103,   122,   103,   129,  1550,
         1482,  1104,  5576, 13335, 15447, 16736,  7162,  1686,  1107,   103,
          117,  1103,  2025,  1276,   119, 22171,   131,  2677,  7541,  1104,
          158,   119,   156,   119,  5696,  7162,   103,  1482,   136,   103,
        26018,   103,  1302,   119,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],

In [61]:
nb_eval_steps

44

In [67]:
len(eval_data)

2767

In [69]:
eval_data[0]

{'input_ids': [101,
  16992,
  125,
  1550,
  1482,
  1150,
  1138,
  1120,
  1655,
  1141,
  6486,
  1150,
  2242,
  1103,
  158,
  119,
  156,
  119,
  103,
  1127,
  1255,
  1107,
  103,
  1244,
  103,
  1105,
  1132,
  158,
  119,
  156,
  103,
  4037,
  1112,
  103,
  1871,
  117,
  2452,
  1106,
  1103,
  2025,
  3303,
  1118,
  1103,
  153,
  5773,
  6098,
  1945,
  119,
  103,
  112,
  188,
  1164,
  1210,
  103,
  103,
  103,
  3555,
  103,
  119,
  126,
  1550,
  1482,
  1104,
  5696,
  7162,
  103,
  1103,
  1244,
  103,
  117,
  2452,
  1106,
  1103,
  2025,
  119,
  3517,
  122,
  119,
  129,
  1550,
  1482,
  1104,
  5576,
  103,
  15447,
  16736,
  7162,
  1686,
  103,
  5224,
  103,
  1103,
  2025,
  1276,
  119,
  22171,
  131,
  2677,
  7541,
  103,
  103,
  119,
  103,
  119,
  103,
  7162,
  1138,
  103,
  136,
  1103,
  26018,
  131,
  1302,
  119,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,


In [59]:
test = {'tokens':[1,2,3]}

In [60]:
tokens = test['tokens'].copy()

In [61]:
tokens[0] = 4

In [62]:
tokens

[4, 2, 3]

In [63]:
test

{'tokens': [1, 2, 3]}

In [4]:
tokenizer.tokenize('RL has been successful in many fields.')

['R', '##L', 'has', 'been', 'successful', 'in', 'many', 'fields', '.']